In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import uuid
import time
from datetime import datetime

options = Options()
options.add_argument("--headless") 
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)

def scrape_kaspi_reviews():
    product_urls = {
        "1": "https://kaspi.kz/shop/p/skin1004-mist-hyalu-cica-cloudy-120-ml-104904631/?0&c=750000000&ref=shared_link",
        "2": "https://kaspi.kz/shop/p/dr-althea-krem-345-relief-dlja-litsa-50-ml-115616909/?0&c=750000000&ref=shared_link",
        "3": "https://kaspi.kz/shop/p/lagom-krem-cellus-mild-moisture-dlja-litsa-80-ml-100676372/?0&c=750000000&ref=shared_link"
    }
    
    all_reviews = []
    
    for product_name, url in product_urls.items():
        print(product_name)
        driver.get(url)
        time.sleep(5)
        
        try:
            show_all_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".reviews__show-all")))
            show_all_button.click()
            time.sleep(3)
        except Exception as e:
            print(e)

        for _ in range(10):  
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            try:
                load_more = driver.find_element(By.CSS_SELECTOR, ".reviews__show-more")
                load_more.click()
                time.sleep(2)
            except:
                pass
        
        soup = BeautifulSoup(driver.page_source, "html.parser")
        reviews = soup.find_all("div", class_="reviews__review")
        print(len(reviews))
        
        for review in reviews[:500]:
            review_id = str(uuid.uuid4())
            
            date_tag = review.find("div", class_="reviews__date")
            raw_date = date_tag.text.strip() if date_tag else None
            try:
                date_obj = datetime.strptime(raw_date, "%d.%m.%Y")
                formatted_date = date_obj.strftime("%Y-%m-%d")
            except:
                formatted_date = None
            
            rating_tag = review.find("div", class_=lambda c: c and "rating" in c)
            rating = None
            if rating_tag:
                classes = rating_tag.get("class")
                for cls in classes:
                    if cls.startswith("_"):
                        try:
                            rating = int(cls[1:]) / 10
                        except:
                            pass
            
            comment_tag = review.find("div", class_="reviews__review-text")
            comment = comment_tag.get_text(strip=True).replace("Комментарий:", "") if comment_tag else None
            
            if comment and any(char.isalpha() for char in comment):
                all_reviews.append({
                    "ID": review_id,
                    "Атауы": product_name,
                    "Күні": formatted_date,
                    "Баға": rating,
                    "Пікір": comment
                })
                if len(all_reviews) % 50 == 0:
                    print({en(all_reviews))
    
    return all_reviews
kaspi_reviews = scrape_kaspi_reviews()

if kaspi_reviews:
    df = pd.DataFrame(kaspi_reviews)
    df = df[["ID", "Атауы", "Күні", "Баға", "Пікір"]]
    
    df.to_csv("kaspi_reviews.csv", index=False, encoding="utf-8-sig")
    print(len(kaspi_reviews))
else:
    print("⚠")

driver.quit()

开始抓取评论数据...
正在抓取产品 1 的评论...
无法点击'显示所有评论'按钮: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF715335335+78597]
	GetHandleVerifier [0x00007FF715335390+78688]
	(No symbol) [0x00007FF7150E91AA]
	(No symbol) [0x00007FF71513F149]
	(No symbol) [0x00007FF71513F3FC]
	(No symbol) [0x00007FF715192467]
	(No symbol) [0x00007FF71516712F]
	(No symbol) [0x00007FF71518F2BB]
	(No symbol) [0x00007FF715166EC3]
	(No symbol) [0x00007FF7151303F8]
	(No symbol) [0x00007FF715131163]
	GetHandleVerifier [0x00007FF7155DEEED+2870973]
	GetHandleVerifier [0x00007FF7155D9698+2848360]
	GetHandleVerifier [0x00007FF7155F6973+2967875]
	GetHandleVerifier [0x00007FF71535017A+188746]
	GetHandleVerifier [0x00007FF71535845F+222255]
	GetHandleVerifier [0x00007FF71533D2B4+111236]
	GetHandleVerifier [0x00007FF71533D462+111666]
	GetHandleVerifier [0x00007FF715323589+5465]
	BaseThreadInitThunk [0x00007FF94D67E8D7+23]
	RtlUserThreadStart [0x00007FF94ED7BF2C+44]

正在加载更多评论...
找到 9 条评论
正在抓取产品 2 的评论...
无法点击'显示所有评论'按钮: Message: 
Stackt